In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import itertools
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

In [92]:
df_modelo = pd.read_csv('df_EDA_predicprecio.csv')
df_modelo.head()

,make,model,version,fuel,year,kms,power,shift,price
0,Opel,Crossland,1.2 GAS 110 GS Line 5p S/S,Gasolina,2022,5.0,110.0,manual,22900
1,Opel,Crossland,1.2 81kW (110CV) GS Line,Gasolina,2022,24847.0,110.0,manual,19990
2,Opel,Crossland,1.5D 88kW (120CV) Business Elegance Auto,Diésel,2021,41356.0,120.0,automatic,18590
3,Opel,Crossland,GS-Line 1.2 GAS MT6 S/S 110cv,Gasolina,2022,11.0,110.0,manual,22700
4,Opel,Crossland,1.2 GS LINE 110 CV 5P,Gasolina,2021,51390.0,110.0,manual,18200


In [93]:
# Función de normalización ajustada
def normalize_version(row):
    version = row['version']
    year = row['year']
    
    # Paso 1: Convertir todo a minúsculas
    version = version.lower()
    
    # Paso 2: Eliminar paréntesis y unidades de potencia como CV, kW
    version = re.sub(r'\s*\(?\d+\s*(cv|kw)\)?\s*', '', version)
    
    # Paso 3: Normalizar palabras clave de combustible
    version = re.sub(r'\bgas\b', 'gasolina', version)
    version = re.sub(r'\bdiesel\b', 'diésel', version)
    
    # Paso 4: Eliminar abreviaturas de transmisión y tipo (MT6, S/S, 5p, auto, manual, etc.)
    version = re.sub(r'\b(mt6|s/s|5p|automatico|manual|auto|edition|line|style|pro|exclusive|gs-line)\b', '', version)
    
    # Paso 5: Eliminar texto que no aporta a la versión, como "cv" o "kw"
    version = re.sub(r'\b(cv|kw)\b', '', version)
    
    # Paso 6: Eliminar caracteres de puntuación innecesarios y espacios adicionales
    version = re.sub(r'\s+', ' ', version)  # Reemplazar múltiples espacios por uno solo
    version = re.sub(r'[^\w\s]', '', version)  # Eliminar caracteres no alfanuméricos (por ejemplo, guiones, paréntesis)
    
    # Paso 7: Eliminar espacios al principio y al final
    version = version.strip()
    
    # Incluir el año en la versión normalizada
    normalized_version = f"{version} {year}"
    
    return normalized_version

# Aplicar la normalización incluyendo el año directamente en df_modelo
df_modelo['normalized_version'] = df_modelo.apply(normalize_version, axis=1)

In [94]:
# Número de filas antes de eliminar duplicados
initial_rows = len(df_modelo)

# Eliminar duplicados basados en 'make', 'model' y 'normalized_version'
df_modelo = df_modelo.drop_duplicates(subset=['make', 'model', 'normalized_version'])

# Número de filas después de eliminar duplicados
final_rows = len(df_modelo)

# Número de filas eliminadas
deleted_rows = initial_rows - final_rows

print(f"\nNúmero de filas originales: {initial_rows}")
print(f"Número de filas después de eliminar duplicados: {final_rows}")
print(f"Número de filas eliminadas: {deleted_rows}")


Número de filas originales: 12453
Número de filas después de eliminar duplicados: 9010
Número de filas eliminadas: 3443


In [95]:
# Función de depuración de valores infinitos
def depurar_valores_infinitos(X):
    """
    Depura valores infinitos separando columnas numéricas y categóricas
    
    Parámetros:
    X (DataFrame): DataFrame de entrada
    
    Retorna:
    DataFrame procesado
    """
    # Separar columnas numéricas y categóricas
    columnas_numericas = X.select_dtypes(include=['int64', 'float64']).columns
    columnas_categoricas = X.select_dtypes(include=['object']).columns
    
    # Clonar el DataFrame
    X_depurado = X.copy()
    
    # Imputar columnas numéricas
    if len(columnas_numericas) > 0:
        # Convertir infinitos a NaN
        X_depurado[columnas_numericas] = X_depurado[columnas_numericas].replace([np.inf, -np.inf], np.nan)
        
        # Imputar valores NaN con la mediana
        imputer_numerico = SimpleImputer(strategy='median')
        X_depurado[columnas_numericas] = imputer_numerico.fit_transform(X_depurado[columnas_numericas])
    
    # Imputar columnas categóricas con el valor más frecuente
    if len(columnas_categoricas) > 0:
        imputer_categorico = SimpleImputer(strategy='most_frequent')
        X_depurado[columnas_categoricas] = imputer_categorico.fit_transform(X_depurado[columnas_categoricas])
    
    return X_depurado

In [96]:
# Función de identificación y manejo de outliers
def identificar_outliers(df, columna):
    """
    Identifica y filtra outliers usando el método del rango intercuartílico
    
    Parámetros:
    df (DataFrame): DataFrame de entrada
    columna (str): Nombre de la columna a analizar
    
    Retorna:
    DataFrame sin outliers
    """
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    return df[(df[columna] >= limite_inferior) & (df[columna] <= limite_superior)]

In [97]:
# Función de creación de características avanzadas
def crear_caracteristicas_avanzadas(df):
    """
    Crea características nuevas y categorías
    
    Parámetros:
    df (DataFrame): DataFrame original
    
    Retorna:
    DataFrame con nuevas características
    """
    df_caracteristicas = df.copy()
    
    # Características existentes
    df_caracteristicas['car_age'] = 2024 - df_caracteristicas['year']
    df_caracteristicas.loc[df_caracteristicas['car_age'] == 0, 'car_age'] = 1
    
    # Nuevas características
    df_caracteristicas['kms_per_year'] = df_caracteristicas['kms'] / df_caracteristicas['car_age']
    df_caracteristicas['power_per_km'] = df_caracteristicas['power'] / df_caracteristicas['kms']
    df_caracteristicas['power_to_kms_ratio'] = df_caracteristicas['power'] / (df_caracteristicas['kms'] + 1)
    
    # Categorías
    df_caracteristicas['age_category'] = pd.cut(
        df_caracteristicas['car_age'], 
        bins=[0, 3, 6, 10, 15, 100], 
        labels=['New', 'Recent', 'Used', 'Old', 'Vintage']
    )
    
    df_caracteristicas['kms_category'] = pd.cut(
        df_caracteristicas['kms'], 
        bins=[0, 50000, 100000, 200000, np.inf], 
        labels=['Low', 'Medium', 'High', 'Very High']
    )
    
    return df_caracteristicas

In [98]:
# Función para plotear importancia de características
def plot_feature_importance(model, feature_names):
    """
    Plotea la importancia de las características
    
    Parámetros:
    model: Modelo entrenado
    feature_names: Nombres de características
    """
    # Obtener importancia de características
    importances = model.named_steps['regressor'].feature_importances_
    
    # Crear DataFrame de importancias
    feature_imp = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    # Visualizar
    plt.figure(figsize=(10, 6))
    sns.barplot(x='importance', y='feature', data=feature_imp.head(10))
    plt.title('Top 10 Features by Importance')
    plt.tight_layout()
    plt.show()

In [99]:
# Paso 1: Preprocesamiento de datos
# Aplicar creación de características avanzadas
df_modelo = crear_caracteristicas_avanzadas(df_modelo)

In [100]:
# Filtrar outliers
df_modelo = identificar_outliers(df_modelo, 'price')

In [101]:
# Depuración inicial de todo el dataset
df_modelo = depurar_valores_infinitos(df_modelo)

In [102]:
# Paso 2: Definir columnas
columnas_categoricas = ['make', 'model', 'version', 'fuel', 'shift', 
                        'age_category', 'kms_category']
columnas_numericas = ['year', 'kms', 'power', 'car_age', 'kms_per_year', 
                      'power_per_km', 'power_to_kms_ratio']

In [103]:
# Paso 3: Preparación de datos
# Separar características y objetivo
X = df_modelo.drop('price', axis=1)
y = df_modelo['price']


In [104]:
# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [105]:
# Paso 4: Preprocesador robusto
preprocessor = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), columnas_numericas),
        ('cat', OneHotEncoder(handle_unknown='ignore'), columnas_categoricas)
    ])

In [106]:
# Paso 5: Definir modelos con hiperparámetros expansivos
modelos = {
    'RandomForest': {
        'modelo': RandomForestRegressor(random_state=42),
        'parametros': {
            'n_estimators': [100, 200, 300, 400],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10, 15],
            'max_features': ['sqrt', 'log2', None],  # 'auto' cambiado a 'sqrt', None
            'bootstrap': [True, False]
        }
    },
    'GradientBoosting': {
        'modelo': GradientBoostingRegressor(random_state=42),
        'parametros': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 4, 5],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'subsample': [0.8, 0.9, 1.0]
        }
    },
    'XGBoost': {
        'modelo': XGBRegressor(random_state=42),
        'parametros': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 4, 5],
            'min_child_weight': [1, 3, 5],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0]
        }
    }
}

In [107]:
# Inicializar un diccionario para guardar los resultados
resultados_modelos = {}

# Depurar valores infinitos una vez
X_train_depurado = depurar_valores_infinitos(X_train)
X_test_depurado = depurar_valores_infinitos(X_test)

In [108]:
# Función para entrenar un único modelo con un conjunto específico de hiperparámetros
def entrenar_modelo_individual(nombre_modelo, modelo_base, params):
    print(f"Entrenando {nombre_modelo} con parámetros: {params}")
    
    # Crear el modelo con los parámetros específicos
    if nombre_modelo == 'RandomForest':
        modelo = RandomForestRegressor(random_state=42, **params)
    elif nombre_modelo == 'GradientBoosting':
        modelo = GradientBoostingRegressor(random_state=42, **params)
    elif nombre_modelo == 'XGBoost':
        modelo = XGBRegressor(random_state=42, **params)
    
    # Crear pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', modelo)
    ])
    
    # Entrenar con validación cruzada
    cv = RepeatedKFold(n_splits=5, n_repeats=3)
    scores = cross_val_score(pipeline, X_train_depurado, y_train, 
                            cv=cv, scoring='neg_mean_squared_error')
    
    # Entrenar con validación cruzada para R2
    r2_scores = cross_val_score(pipeline, X_train_depurado, y_train, 
                            cv=cv, scoring='r2')
    
    # Entrenar en todo el conjunto de entrenamiento para guardar el modelo final
    pipeline.fit(X_train_depurado, y_train)
    
     # Evaluar en el conjunto de prueba
    y_pred = pipeline.predict(X_test_depurado)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return {
        'pipeline': pipeline,
        'cv_mse_mean': mse_scores.mean(),
        'cv_mse_std': mse_scores.std(),
        'cv_r2_mean': r2_scores.mean(),
        'cv_r2_std': r2_scores.std(),
        'test_mse': mse,
        'test_r2': r2,
        'params': params
    }
    
    

In [109]:
# Función para generar todas las combinaciones posibles de hiperparámetros
def generar_combinaciones_parametros(parametros):
    claves = list(parametros.keys())
    valores = list(parametros.values())
    
    combinaciones = []
    for combo in itertools.product(*valores):
        combinacion = {claves[i]: combo[i] for i in range(len(claves))}
        combinaciones.append(combinacion)
    
    return combinaciones

In [ ]:
# Entrenar modelos uno por uno
for nombre, config in modelos.items():
    print(f"\n{'='*50}")
    print(f"Iniciando entrenamiento de {nombre}")
    print(f"{'='*50}")
    
    resultados_modelos[nombre] = []
    
    # Generar todas las combinaciones de hiperparámetros
    combinaciones = generar_combinaciones_parametros(config['parametros'])
    
    print(f"Se entrenarán {len(combinaciones)} combinaciones de hiperparámetros para {nombre}")
    
    # Entrenar cada combinación individualmente
    for i, params in enumerate(combinaciones):
        print(f"\nCombinación {i+1}/{len(combinaciones)}")
        try:
            resultado = entrenar_modelo_individual(nombre, config['modelo'], params)
            resultados_modelos[nombre].append(resultado)
            
            # Mostrar resultados parciales con R2
            print(f"MSE en validación cruzada: {-resultado['cv_mse_mean']:.4f} ± {resultado['cv_mse_std']:.4f}")
            print(f"R² en validación cruzada: {resultado['cv_r2_mean']:.4f} ± {resultado['cv_r2_std']:.4f}")
            print(f"MSE en conjunto de prueba: {resultado['test_mse']:.4f}")
            print(f"R² en conjunto de prueba: {resultado['test_r2']:.4f}")
            
        except Exception as e:
            print(f"Error con esta combinación: {str(e)}")
            print("Saltando a la siguiente combinación...")
            continue
    
    # Verificar si hay resultados para este modelo
    if not resultados_modelos[nombre]:
        print(f"No se obtuvieron resultados válidos para {nombre}. Pasando al siguiente modelo.")
        continue
    


Iniciando entrenamiento de RandomForest
Se entrenarán 384 combinaciones de hiperparámetros para RandomForest

Combinación 1/384
Entrenando RandomForest con parámetros: {'n_estimators': 100, 'max_depth': None, 'min_samples_split': 2, 'max_features': 'sqrt', 'bootstrap': True}
Error con esta combinación: name 'mse_scores' is not defined
Saltando a la siguiente combinación...

Combinación 2/384
Entrenando RandomForest con parámetros: {'n_estimators': 100, 'max_depth': None, 'min_samples_split': 2, 'max_features': 'sqrt', 'bootstrap': False}


In [ ]:
# Identificar el mejor modelo para este tipo (basado en R2 de prueba)
mejor_indice = np.argmax([res['test_r2'] for res in resultados_modelos[nombre]])
mejor_modelo = resultados_modelos[nombre][mejor_indice]
    
print(f"\n{'*'*50}")
print(f"Mejor configuración para {nombre}:")
print(f"Parámetros: {mejor_modelo['params']}")
print(f"MSE en validación cruzada: {-mejor_modelo['cv_mse_mean']:.4f}")
print(f"R² en validación cruzada: {mejor_modelo['cv_r2_mean']:.4f}")
print(f"MSE en conjunto de prueba: {mejor_modelo['test_mse']:.4f}")
print(f"R² en conjunto de prueba: {mejor_modelo['test_r2']:.4f}")
print(f"{'*'*50}")

In [ ]:
# Verificar si se obtuvieron resultados para algún modelo
modelos_con_resultados = {k: v for k, v in resultados_modelos.items() if v}

if not modelos_con_resultados:
    print("No se obtuvieron resultados válidos para ningún modelo.")
else:
    # Encontrar el mejor modelo entre todos los tipos (basado en R2 de prueba)
    mejor_tipo = max(modelos_con_resultados.keys(), 
                    key=lambda k: max(res['test_r2'] for res in modelos_con_resultados[k]))

    mejor_indice_global = np.argmax([res['test_r2'] for res in modelos_con_resultados[mejor_tipo]])
    mejor_modelo_global = modelos_con_resultados[mejor_tipo][mejor_indice_global]

    print(f"\n{'#'*50}")
    print(f"MEJOR MODELO GLOBAL: {mejor_tipo}")
    print(f"Parámetros: {mejor_modelo_global['params']}")
    print(f"MSE en validación cruzada: {-mejor_modelo_global['cv_mse_mean']:.4f}")
    print(f"R² en validación cruzada: {mejor_modelo_global['cv_r2_mean']:.4f}")
    print(f"MSE en conjunto de prueba: {mejor_modelo_global['test_mse']:.4f}")
    print(f"R² en conjunto de prueba: {mejor_modelo_global['test_r2']:.4f}")
    print(f"{'#'*50}")

In [ ]:
# Paso 8: Ensemble de Modelos
# Crear el ensemble
# Suponiendo que 'resultados_modelos' contiene los resultados de grid search
ensemble_modelos = [
    ('rf', modelos['RandomForest']['modelo'].set_params(**resultados_modelos['RandomForest']['Modelo'].best_params_)),
    ('gb', modelos['GradientBoosting']['modelo'].set_params(**resultados_modelos['GradientBoosting']['Modelo'].best_params_)),
    ('xgb', modelos['XGBoost']['modelo'].set_params(**resultados_modelos['XGBoost']['Modelo'].best_params_))
]

In [ ]:
ensemble = VotingRegressor(ensemble_modelos)

In [ ]:
# Pipeline para el ensemble
ensemble_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', ensemble)
])

In [ ]:
# Entrenar ensemble
ensemble_pipeline.fit(X_train_depurado, y_train)

In [ ]:
# Predicciones del ensemble
predicciones_ensemble = ensemble_pipeline.predict(X_test_depurado)

# Métricas del ensemble
mse_ensemble = mean_squared_error(y_test, predicciones_ensemble)
rmse_ensemble = np.sqrt(mse_ensemble)
mae_ensemble = mean_absolute_error(y_test, predicciones_ensemble)
r2_ensemble = r2_score(y_test, predicciones_ensemble)

print("\nEnsemble Model Results:")
print(f"R² Score: {r2_ensemble:.4f}")
print(f"RMSE: {rmse_ensemble:.2f}")
print(f"MAE: {mae_ensemble:.2f}")

In [ ]:
# Paso 10: Importancia de características
plot_feature_importance(mejor_modelo, 
    preprocessor.get_feature_names_out())